In [ ]:
!pip install transformers datasets torch

In [13]:
import torch.nn as nn

In [ ]:
# From a new Colab cell:
!pip install --upgrade datasets transformers huggingface_hub gcsfs fsspec

In [3]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments
from datasets import load_dataset
import torch

In [4]:
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, Trainer, TrainingArguments
from datasets import load_dataset

# Load the SQuAD dataset
dataset = load_dataset("squad")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.62k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [6]:
# 2. Define teacher and student models
teacher_name = "bert-base-uncased"
student_name = "distilbert-base-uncased"

# Initialize tokenizer and models
tokenizer = AutoTokenizer.from_pretrained(teacher_name)
teacher_model = AutoModelForQuestionAnswering.from_pretrained(teacher_name)
student_model = AutoModelForQuestionAnswering.from_pretrained(student_name)

Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# 3. Preprocessing function for question answering
max_length = 384
doc_stride = 128

def prepare_features(examples):
    tokenized = tokenizer(
        examples['question'], examples['context'],
        truncation='only_second', max_length=max_length,
        stride=doc_stride, return_overflowing_tokens=True,
        return_offsets_mapping=True, padding='max_length'
    )
    sample_mapping = tokenized.pop('overflow_to_sample_mapping')
    offset_mapping = tokenized.pop('offset_mapping')

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        input_ids = tokenized['input_ids'][i]
        sequence_ids = tokenized.sequence_ids(i)
        sample_index = sample_mapping[i]
        answers = examples['answers'][sample_index]

        if len(answers['answer_start']) == 0:
            start_positions.append(0)
            end_positions.append(0)
        else:
            start_char = answers['answer_start'][0]
            end_char = start_char + len(answers['text'][0])
            # Locate start and end token indices
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # If the answer is not fully in the span, label as 0,0
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                start_positions.append(0)
                end_positions.append(0)
            else:
                while token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char:
                    token_start_index += 1
                start_positions.append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                end_positions.append(token_end_index + 1)

    tokenized['start_positions'] = start_positions
    tokenized['end_positions'] = end_positions
    return tokenized

# Tokenize the dataset
tokenized_datasets = dataset.map(
    prepare_features,
    batched=True,
    remove_columns=dataset['train'].column_names
)

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [10]:
# 4. Custom Trainer for distillation
class DistillationTrainer(Trainer):
    def __init__(self, teacher_model, alpha=0.5, temperature=2.0, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model
        self.alpha = alpha
        self.temperature = temperature
        self.kl_loss = nn.KLDivLoss(reduction='batchmean')

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = {
            'start_positions': inputs.pop('start_positions'),
            'end_positions': inputs.pop('end_positions')
        }
        # Student forward pass
        outputs_student = model(**inputs, **labels)
        loss_ce = outputs_student.loss

        # Teacher forward (no gradients)
        with torch.no_grad():
            outputs_teacher = self.teacher_model(**inputs)

        t = self.temperature
        # Distillation losses
        loss_start = self.kl_loss(
            nn.functional.log_softmax(outputs_student.start_logits / t, dim=-1),
            nn.functional.softmax(outputs_teacher.start_logits / t, dim=-1)
        ) * (t * t)
        loss_end = self.kl_loss(
            nn.functional.log_softmax(outputs_student.end_logits / t, dim=-1),
            nn.functional.softmax(outputs_teacher.end_logits / t, dim=-1)
        ) * (t * t)
        loss_distill = (loss_start + loss_end) / 2

        # Combine CE and distillation losses
        loss = self.alpha * loss_distill + (1 - self.alpha) * loss_ce
        return (loss, outputs_student) if return_outputs else loss

In [11]:
# 5. Set up training arguments (without evaluation_strategy for compatibility)
training_args = TrainingArguments(
    output_dir='./models_distilled',
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    save_total_limit=2,
    logging_steps=100,
    report_to=[]
)

In [14]:
# 6. Initialize the distillation trainer
trainer = DistillationTrainer(
    teacher_model=teacher_model,
    model=student_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer
)

<ipython-input-10-b424c080afc1>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `DistillationTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


In [ ]:
# 4. Custom Trainer for distillation
class DistillationTrainer(Trainer):
    def __init__(self, teacher_model, alpha=0.5, temperature=2.0, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.teacher_model = teacher_model
        self.alpha = alpha
        self.temperature = temperature
        # Ensure teacher is on the same device as student
        self.teacher_model.to(self.args.device)
        self.kl_loss = nn.KLDivLoss(reduction='batchmean')

    def compute_loss(self, model, inputs, return_outputs=False, **kwargs):
        labels = {
            'start_positions': inputs.pop('start_positions'),
            'end_positions': inputs.pop('end_positions')
        }
        # Student forward pass
        outputs_student = model(**inputs, **labels)
        loss_ce = outputs_student.loss

        # Teacher forward (no gradients)
        with torch.no_grad():
            outputs_teacher = self.teacher_model(**inputs)

        t = self.temperature
        # Distillation losses
        loss_start = self.kl_loss(
            nn.functional.log_softmax(outputs_student.start_logits / t, dim=-1),
            nn.functional.softmax(outputs_teacher.start_logits / t, dim=-1)
        ) * (t * t)
        loss_end = self.kl_loss(
            nn.functional.log_softmax(outputs_student.end_logits / t, dim=-1),
            nn.functional.softmax(outputs_teacher.end_logits / t, dim=-1)
        ) * (t * t)
        loss_distill = (loss_start + loss_end) / 2

        # Combine CE and distillation losses
        loss = self.alpha * loss_distill + (1 - self.alpha) * loss_ce
        return (loss, outputs_student) if return_outputs else loss

# 5. Set up training arguments (without evaluation_strategy for compatibility)
training_args = TrainingArguments(
    output_dir='./models_distilled',
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    save_total_limit=2,
    logging_steps=100,
    report_to=[]
)

# 6. Initialize the distillation trainer
trainer = DistillationTrainer(
    teacher_model=teacher_model,
    model=student_model,
    args=training_args,
    train_dataset=tokenized_datasets['train'],
    eval_dataset=tokenized_datasets['validation'],
    tokenizer=tokenizer
)

# 7. Train and save the distilled student model
trainer.train()
trainer.save_model('./distilled_student_model')

# 8. Print compression statistics
def print_compression_stats(teacher, student):
    teacher_params = sum(p.numel() for p in teacher.parameters())
    student_params = sum(p.numel() for p in student.parameters())
    print(f"Teacher parameters: {teacher_params}")
    print(f"Student parameters: {student_params}")
    print(f"Compression ratio (teacher/student): {teacher_params / student_params:.2f}x")

print_compression_stats(teacher_model, student_model)

<ipython-input-21-9149744dbc39>:4: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `DistillationTrainer.__init__`. Use `processing_class` instead.
  super().__init__(*args, **kwargs)


Step,Training Loss
100,2.421000
200,1.878100
300,1.593600
400,1.416200
500,1.360000
600,1.297900
700,1.260900
800,1.268100
900,1.206800
1000,1.164400


In [ ]:
infer_tokenizer = AutoTokenizer.from_pretrained('./distilled_student_model')
infer_model = AutoModelForQuestionAnswering.from_pretrained('./distilled_student_model')
qa_pipeline = pipeline('question-answering', model=infer_model, tokenizer=infer_tokenizer)

# Example usage
test_context = "The Transformer architecture was introduced in the paper Attention is All You Need in 2017."
test_question = "When was the Transformer architecture introduced?"
result = qa_pipeline(question=test_question, context=test_context)
print("Answer:", result['answer'])
print("Score:", result['score'])